#패키지 불러오기

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
% matplotlib inline
from datetime import datetime
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix
from sklearn.utils import class_weight
from sklearn.preprocessing import RobustScaler
from sklearn.neighbors import KNeighborsClassifier

#데이터 불러오기 및 전처리

DAYS_BIRTH: 출생일

* 데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 하루 전에 태어났음을 의미 추가

DAYS_EMPLOYED: 업무 시작일
* 데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 하루 전부터 일을 시작함을 의미양수 값은 고용되지 않은 상태를 의미함

begin_month: 신용카드 발급 월

* 데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 한 달 전에 신용카드를 발급함을 의미

In [2]:
data_train = pd.read_csv('/content/train.csv')

In [3]:
data_train = data_train.fillna('unknown')

X = data_train.drop(columns=['index', 'FLAG_MOBIL', 'child_num', 'credit'])
y = data_train['credit']

X_encoded = pd.get_dummies(X, columns=['gender', 'car', 'reality', 'income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type'])

In [4]:
X_encoded

,income_total,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,family_size,begin_month,gender_F,gender_M,car_N,car_Y,reality_N,reality_Y,income_type_Commercial associate,income_type_Pensioner,income_type_State servant,income_type_Student,income_type_Working,edu_type_Academic degree,edu_type_Higher education,edu_type_Incomplete higher,edu_type_Lower secondary,edu_type_Secondary / secondary special,family_type_Civil marriage,family_type_Married,family_type_Separated,family_type_Single / not married,family_type_Widow,house_type_Co-op apartment,house_type_House / apartment,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,occyp_type_Accountants,occyp_type_Cleaning staff,occyp_type_Cooking staff,occyp_type_Core staff,occyp_type_Drivers,occyp_type_HR staff,occyp_type_High skill tech staff,occyp_type_IT staff,occyp_type_Laborers,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,occyp_type_unknown
0,202500.0,-13899,-4709,0,0,0,2.0,-6.0,1,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,247500.0,-11380,-1540,0,0,1,3.0,-5.0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,450000.0,-19087,-4434,0,1,0,2.0,-22.0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,202500.0,-15088,-2092,0,1,0,2.0,-37.0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,157500.0,-15037,-2105,0,0,0,2.0,-26.0,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,225000.0,-12079,-1984,0,0,0,4.0,-2.0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26453,180000.0,-15291,-2475,0,0,0,2.0,-47.0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
26454,292500.0,-10082,-2015,0,0,0,2.0,-25.0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26455,171000.0,-10145,-107,0,0,0,1.0,-59.0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [5]:
num_features = [(0, 'income_total'), (1, 'DAYS_BIRTH'), (2, 'DAYS_EMPLOYED'), (6, 'family_size'), (7, 'begin_month')]
already = [0, 1, 2, 6, 7]
features = list(range(54))

for i in already:
  features.remove(i)

scaler = RobustScaler()
X_dummy = X_encoded.iloc[:, features].reset_index(drop=True)
X_robust = pd.DataFrame(scaler.fit_transform(X_encoded.iloc[:, [t[0] for t in num_features]]), columns=[t[1] for t in num_features])
X_final = pd.concat([X_robust,X_dummy ], axis=1)

In [6]:
X_final

,income_total,DAYS_BIRTH,DAYS_EMPLOYED,family_size,begin_month,work_phone,phone,email,gender_F,gender_M,car_N,car_Y,reality_N,reality_Y,income_type_Commercial associate,income_type_Pensioner,income_type_State servant,income_type_Student,income_type_Working,edu_type_Academic degree,edu_type_Higher education,edu_type_Incomplete higher,edu_type_Lower secondary,edu_type_Secondary / secondary special,family_type_Civil marriage,family_type_Married,family_type_Separated,family_type_Single / not married,family_type_Widow,house_type_Co-op apartment,house_type_House / apartment,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,occyp_type_Accountants,occyp_type_Cleaning staff,occyp_type_Cooking staff,occyp_type_Core staff,occyp_type_Drivers,occyp_type_HR staff,occyp_type_High skill tech staff,occyp_type_IT staff,occyp_type_Laborers,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,occyp_type_unknown
0,0.434783,0.235934,-1.154406,0.0,0.666667,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0.869565,0.596564,-0.000364,1.0,0.703704,0,0,1,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,2.826087,-0.506800,-1.054261,0.0,0.074074,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0.434783,0.065712,-0.201384,0.0,-0.481481,0,1,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0.000000,0.073014,-0.206118,0.0,-0.074074,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,0.652174,0.496492,-0.162054,2.0,0.814815,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26453,0.217391,0.036650,-0.340859,0.0,-0.851852,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
26454,1.304348,0.782391,-0.173343,0.0,-0.037037,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26455,0.130435,0.773372,0.521486,-1.0,-1.296296,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [7]:
X_train, X_valid, y_train, y_valid = train_test_split(X_final, y, test_size=0.2, random_state=42)

In [8]:
set = [X_train, X_valid, y_train, y_valid]
for i in set:
  print(i.shape)

(21165, 54)
(5292, 54)
(21165,)
(5292,)


In [9]:
X_train

,income_total,DAYS_BIRTH,DAYS_EMPLOYED,family_size,begin_month,work_phone,phone,email,gender_F,gender_M,car_N,car_Y,reality_N,reality_Y,income_type_Commercial associate,income_type_Pensioner,income_type_State servant,income_type_Student,income_type_Working,edu_type_Academic degree,edu_type_Higher education,edu_type_Incomplete higher,edu_type_Lower secondary,edu_type_Secondary / secondary special,family_type_Civil marriage,family_type_Married,family_type_Separated,family_type_Single / not married,family_type_Widow,house_type_Co-op apartment,house_type_House / apartment,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,occyp_type_Accountants,occyp_type_Cleaning staff,occyp_type_Cooking staff,occyp_type_Core staff,occyp_type_Drivers,occyp_type_HR staff,occyp_type_High skill tech staff,occyp_type_IT staff,occyp_type_Laborers,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,occyp_type_unknown
23445,1.086957,0.324409,0.446832,1.0,0.407407,0,1,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
13278,0.608696,0.059699,0.498543,0.0,0.074074,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9303,0.217391,0.484753,0.514202,2.0,-0.407407,1,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7794,1.956522,0.696063,-0.117990,2.0,0.037037,1,0,0,1,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
10507,0.217391,0.253830,-0.416970,-1.0,-0.888889,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21575,-0.217391,0.101503,0.000000,3.0,-0.851852,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
5390,-0.847826,0.770222,0.258558,1.0,-0.814815,1,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
860,-0.956522,-0.731568,133.569556,0.0,0.444444,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
15795,-0.434783,0.821188,0.245812,1.0,-0.407407,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


submission용 test set 전처리

In [10]:
data_test = pd.read_csv('/content/test.csv')

In [11]:
data_test = data_test.fillna('unknown')

X_test = data_test.drop(columns=['index', 'FLAG_MOBIL', 'child_num'])

X_test_encoded = pd.get_dummies(X_test, columns=['gender', 'car', 'reality', 'income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type'])

In [12]:
X_test_encoded

,income_total,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,family_size,begin_month,gender_F,gender_M,car_N,car_Y,reality_N,reality_Y,income_type_Commercial associate,income_type_Pensioner,income_type_State servant,income_type_Student,income_type_Working,edu_type_Academic degree,edu_type_Higher education,edu_type_Incomplete higher,edu_type_Lower secondary,edu_type_Secondary / secondary special,family_type_Civil marriage,family_type_Married,family_type_Separated,family_type_Single / not married,family_type_Widow,house_type_Co-op apartment,house_type_House / apartment,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,occyp_type_Accountants,occyp_type_Cleaning staff,occyp_type_Cooking staff,occyp_type_Core staff,occyp_type_Drivers,occyp_type_HR staff,occyp_type_High skill tech staff,occyp_type_IT staff,occyp_type_Laborers,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,occyp_type_unknown
0,112500.0,-21990,365243,0,1,0,2.0,-60.0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,135000.0,-18964,-8671,0,1,0,2.0,-36.0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,69372.0,-15887,-217,1,1,0,2.0,-40.0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,112500.0,-19270,-2531,1,0,0,2.0,-41.0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,225000.0,-17822,-9385,1,0,0,2.0,-8.0,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,202500.0,-18593,-5434,1,1,0,2.0,-19.0,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9996,202500.0,-10886,-1315,1,0,0,2.0,-34.0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
9997,292500.0,-21016,-14018,0,0,0,2.0,-55.0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
9998,180000.0,-16541,-1085,0,1,0,2.0,-33.0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [13]:
num_features = [(0, 'income_total'), (1, 'DAYS_BIRTH'), (2, 'DAYS_EMPLOYED'), (6, 'family_size'), (7, 'begin_month')]
already = [0, 1, 2, 6, 7]
features = list(range(54))

for i in already:
  features.remove(i)

scaler = RobustScaler()
X_test_dummy = X_test_encoded.iloc[:, features].reset_index(drop=True)
X_test_robust = pd.DataFrame(scaler.fit_transform(X_test_encoded.iloc[:, [t[0] for t in num_features]]), columns=[t[1] for t in num_features])
X_test_final = pd.concat([X_test_robust,X_test_dummy ], axis=1)

# classification

naive bayes = 음수 조정 필요

RandomForest

In [14]:
from sklearn.model_selection import GridSearchCV as cv

In [15]:
from sklearn.ensemble import RandomForestClassifier

In [16]:
rf_params = {'n_estimators':range(10,50,10) ,'criterion':('gini', 'entropy'), 'max_depth':range(10,50,10),'max_features':('auto', 'log2', 'sqrt') }

In [17]:
clf_rf = GridSearchCV(RandomForestClassifier(), param_grid=rf_params)

In [ ]:
clf_rf.fit(X_train,y_train)

In [ ]:
clf_rf.best_params_

In [ ]:
y_pred_rf = clf_rf.predict_proba(X_valid)

In [ ]:
log_loss(y_valid, y_pred_rf)

In [ ]:
curr_time = datetime.now().strftime('%y%m%d')
X_raw = pd.read_csv('/content/test.csv', encoding='utf-8')
l_idx = X_raw['index']
submission = pd.DataFrame(np.insert(clf_rf.predict_proba(X_test_final), 0, l_idx, axis=1), columns=['index', 0, 1, 2])
submission['index'] = submission['index'].astype('int')
submission.to_csv(f'submission_{curr_time}_rf.csv', index=False)

ADABoost

In [19]:
from sklearn.ensemble import AdaBoostClassifier

In [22]:
ada_params = {'n_estimators':range(10,50,10), 'learning_rate':(0.01, 0.1, 1)}

In [23]:
clf_ada = GridSearchCV(AdaBoostClassifier(), param_grid = ada_params)
clf_ada.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                          base_estimator=None,
                                          learning_rate=1.0, n_estimators=50,
                                          random_state=None),
             iid='deprecated', n_jobs=None,
             param_grid={'learning_rate': (0.01, 0.1, 1),
                         'n_estimators': range(10, 50, 10)},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [24]:
clf_ada.best_params_

{'learning_rate': 1, 'n_estimators': 10}

In [25]:
y_pred_ada = clf_ada.predict_proba(X_valid)

In [26]:
log_loss(y_true=y_valid, y_pred=y_pred_ada)

1.0315305445121152

In [27]:
curr_time = datetime.now().strftime('%y%m%d')
X_raw = pd.read_csv('/content/test.csv', encoding='utf-8')
l_idx = X_raw['index']
submission = pd.DataFrame(np.insert(clf_ada.predict_proba(X_test_final), 0, l_idx, axis=1), columns=['index', 0, 1, 2])
submission['index'] = submission['index'].astype('int')

In [28]:
submission.to_csv(f'submission_{curr_time}_ada.csv', index=False)

In [29]:
y_pred_ada = clf_ada.predict(X_valid)

In [30]:
accuracy_score(y_true=y_valid, y_pred=y_pred_ada)

0.6880196523053665

XGB

In [31]:
from xgboost import plot_importance
from xgboost import XGBClassifier

In [32]:
xgb_params = {'eta':(0.01, 0.1, 1), 'max_depth':range(10,50,10)}

In [43]:
clf_xgb = GridSearchCV(XGBClassifier(), param_grid=xgb_params)

In [ ]:
clf_xgb.fit(X_train, y_train)

In [ ]:
clf_xgb.best_params

In [ ]:
y_pred_xgb = clf_xgb.predict_proba(X_valid)
log_loss(y_true=y_valid, y_pred=y_pred_xgb)

In [ ]:
curr_time = datetime.now().strftime('%y%m%d')
X_raw = pd.read_csv('/content/test.csv', encoding='utf-8')
l_idx = X_raw['index']
submission = pd.DataFrame(np.insert(clf_xgb.predict_proba(X_test_final), 0, l_idx, axis=1), columns=['index', 0, 1, 2])
submission['index'] = submission['index'].astype('int')
submission.to_csv(f'submission_{curr_time}_xgb.csv', index=False)

In [ ]:
y_pred_xgb = clf_xgb.predict(X_valid)
accuracy_score(y_true=y_valid, y_pred=y_pred_xgb)

SVC

In [ ]:
svc_params = {'kernel':('linear', 'poly', 'rbf', 'sigmoid', 'precomputed'), 'C':[1,10]}

In [ ]:
clf_svr=GridSearchCV(SVC(probability=True),param_grid=svc_params )

In [ ]:
clf_svc.fit(X_train, y_train)

In [ ]:
y_pred_svc = clf_svc.predict_proba(X_valid)

In [ ]:
log_loss(y_true=y_valid, y_pred=y_pred_svr)

In [ ]:
curr_time = datetime.now().strftime('%y%m%d')
X_raw = pd.read_csv('/content/test.csv', encoding='utf-8')
l_idx = X_raw['index']
submission = pd.DataFrame(np.insert(svm_clf.predict_proba(X_test_final), 0, l_idx, axis=1), columns=['index', 0, 1, 2])
submission['index'] = submission['index'].astype('int')
submission.to_csv(f'submission_{curr_time}_svc.csv', index=False)